In [36]:
import numpy as np
import pandas as pd
import pylab as plt
import seaborn as sns
import pydotplus
from matplotlib import pyplot as plt
from sklearn.datasets import load_iris
%matplotlib inline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier

In [73]:
from PIL import Image
from io import StringIO
from sklearn.tree import export_graphviz
import io

In [5]:
iris = load_iris()

In [3]:
df = pd.DataFrame(load_iris().data)

In [15]:
df['target'] = iris.target

In [16]:
df.columns = iris.feature_names

In [27]:
names = {x: y for x, y in zip(range(0,3), iris.target_names)}

In [32]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [33]:
y = df['target']

In [34]:
X = df.drop('target', axis=1)

In [35]:
X_train, X_hold, y_train, y_hold = train_test_split(X, y, 
                                                      test_size=0.3, 
                                                      random_state=17)

In [38]:
tree = DecisionTreeClassifier(random_state=17)

In [40]:
np.mean(cross_val_score(tree, X_train, y_train, cv=5))
# Вспомнить что такое cross_val_score

0.8939130434782608

In [42]:
from sklearn.model_selection import GridSearchCV

In [49]:
tree_params = {'max_depth': np.arange(1, 11), 'max_features':[.1, .2, .3, .4]}

In [50]:
tree_grid = GridSearchCV(tree, tree_params, cv=5, n_jobs=-1)

In [51]:
%%time
tree_grid.fit(X_train, y_train)

Wall time: 259 ms


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=17,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                         'max_features': [0.1, 0.2, 0

In [54]:
tree_grid.best_params_


{'max_depth': 5, 'max_features': 0.1}

In [55]:
tree_valid_pred = tree_grid.predict(X_hold)

In [56]:
from sklearn.metrics import accuracy_score

In [57]:
accuracy_score(y_hold, tree_valid_pred)

0.9777777777777777

## tree_grid.best_estimator_ - так можно получить дерево из решётки

In [77]:
dot_data = StringIO()
export_graphviz(tree_grid.best_estimator_, feature_names=iris.feature_names,
               out_file=dot_data, filled=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
img = Image.open(io.BytesIO(graph.create_png()))
img.save('../../jupyter_russian/topic03_decision_trees_knn/iris2.png', format='png')

<img src="iris2.png">

## Чисто для проверки сделаем дерево без настройки решётки (Кроссвалидации)

In [61]:
second_tree = DecisionTreeClassifier(max_depth=5, max_features=0.1).fit(X_train, y_train)
second_tree.score(X_hold, y_hold)

0.9555555555555556

In [60]:


dot_data = StringIO()
export_graphviz(second_tree, feature_names=iris.feature_names,
               out_file=dot_data, filled=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
img = Image.open(io.BytesIO(graph.create_png()))
img.save('../../jupyter_russian/topic03_decision_trees_knn/iris.png', format='png')

<img src="iris.png">